In [24]:
import pandas as pd
import numpy as np
import seaborn as sns

import requests
import json

In [4]:
data = pd.read_csv('despesas-entre-2003-e-2022.csv', encoding='latin1', sep=';')

In [8]:
data['SUBELEMENTO DE DESPESA'] = data['SUBELEMENTO DE DESPESA'].astype('category')

In [11]:
data

,DATA PGTO,CPF SERVIDOR,CPF/CNPJ FORNECEDOR,NOME FORNECEDOR,VALOR,TIPO,SUBELEMENTO DE DESPESA,CDIC
0,02/01/2003,***.010.008-**,3.134920e+13,JW TRANSPORTADORA TURÖSTICA,"R$ 1.696,90",D,LOCACAO DE MEIOS DE TRANSPORTE,00200.220539/2003-63.R.08.18/12/2002.31/12/2014.N
1,02/01/2003,***.010.008-**,3.134920e+13,JW TRANSPORTADORA TURÖSTICA,"R$ 1.336,60",D,LOCACAO DE MEIOS DE TRANSPORTE,00200.220539/2003-63.R.08.18/12/2002.31/12/2014.N
2,02/01/2003,***.868.251-**,3.134920e+13,JW TRANSPORTADORA TURÖSTICA,"R$ 1.396,43",D,LOCACAO DE MEIOS DE TRANSPORTE,00200.600539/2003-63.R.08.18/12/2002.31/12/2014.N
3,03/01/2003,***.004.131-**,5.927170e+11,FLORES ALVORADA,"R$ 8.585,00",C,OUTROS SERVIOS DE TERCEIROS-PESSOA JURÖDICA,00200.800212/2003-91.R.08.07/12/2002.31/12/2014.N
4,03/01/2003,***.004.131-**,8.393080e+11,CRISTAL LIMPEZA - COMRCIO DE MATERIAIS PARA L...,"R$ 251,50",C,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZAÇO,00200.800212/2003-91.R.08.07/12/2002.31/12/2014.N
...,...,...,...,...,...,...,...,...
113337,04/12/2022,***.136.591-**,4.750841e+13,COMPANHIA BRASILEIRA DE DISTRIBUIÇO,"518,07",C,GENEROS DE ALIMENTAÇO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N
113338,04/12/2022,***.136.591-**,1.255044e+13,RESTAURANTE SOHO,"330,52",C,FORNECIMENTO DE ALIMENTAÇO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N
113339,04/12/2022,***.136.591-**,3.696869e+12,BIG TRANS COMERCIAL DE ALIMENTOS LTDA,"54,66",C,GENEROS DE ALIMENTAÇO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N
113340,Fonte: SUPRIM,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
def get_company_info(cnpj):
    url = f"https://receitaws.com.br/v1/cnpj/%7Bcnpj%7D"
    querystring = {"token":"XXXXXXXX-XXXX-XXXX-XXXX-XXXXXXXXXXXX","cnpj":"06990590000123","plugin":"RF"}
    response = requests.request("GET", url, params=querystring)
    return json.loads(response.text)

def print_company_adress(info):
    print(f"{info['nome']}: {info['logradouro']}, {info['numero']} ({info['complemento']}) - CEP {info['cep']}")

GOOGLE_CNPJ = "06990590000123"
info = get_company_info(GOOGLE_CNPJ)
print_company_adress(info)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [38]:
info = get_company_info(GOOGLE_CNPJ)
print(info)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [21]:
get_company_info(str(int(data['CPF/CNPJ FORNECEDOR'].iloc[0])))

NameError: name 'get_company_info' is not defined